# Master Discogs Database

In [22]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collection import collections
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep


################################################################################
## Music Stuff
################################################################################

### Master DB code
from masterdb import discConv, isKnownAlbum, directoryName
from masterdb import getSlimArtistDB, getArtistNameToIDMap
from masterdb import getMusicData, getRowByIndex

### General
from discogsBase import discogs

### Discogs
from artists import artists
from artist import artist
from discogsUtils import discogsUtils
from albums import albums
from album import album, albumURLInfo

### AllMusic
from artistsAM import artistsAM
from artistAM import artistAM
from discogsUtils import allmusicUtils

### MusicBrainz
from artistsMB import artistsMB
from artistMB import artistMB
from discogsUtils import musicbrainzUtils

## AceBootlegs
from artistAB import artistAB
from artistsAB import artistsAB
from discogsUtils import acebootlegsUtils

## RateYourMusic
from artistRM import artistRM
from artistsRM import artistsRM
from discogsUtils import rateyourmusicUtils

## LastFM
from artistLM import artistLM
from artistsLM import artistsLM
from discogsUtils import lastfmUtils


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-03-27 16:12:34.916310


In [10]:
%load_ext autoreload
%autoreload

discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)
dutilsAM = allmusicUtils()

discDC   = discogs('discogs')
artsDC   = artists(discDC)
dutilsDC = discogsUtils()

discMB   = discogs('musicbrainz')
artsMB   = artistsMB(discMB)
dutilsMB = musicbrainzUtils()

discAB   = discogs('acebootlegs')
artsAB   = artistsAB(discAB)
dutilsAB = acebootlegsUtils()

discRM   = discogs('rateyourmusic')
artsRM   = artistsRM(discRM)
dutilsRM = rateyourmusicUtils()

discLM   = discogs('lastfm')
artsLM   = artistsLM(discLM)
dutilsLM = lastfmUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

#### AllMusic

In [32]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discAM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistAMDB = getSlimArtistDB(discAM, force=True)
discSlimArtistAMDB.head()


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistAMAlbumsDB = getArtistAlbumsDB(discAM, force=True)
discArtistAMAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Mar 27, 2020 16:35:19 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/0-Metadata.p  	0 382
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/1-Metadata.p  	1 746
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/10-Metadata.p  	2 1163
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/11-Metadata.p  	3 1537
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/12-Metadata.p  	4 1940
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/13-Metadata.p  	5 2322
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/14-Metadata.p  	6 2681
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/15-Metadata.p  	7 3039
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/16-Metadata.p  	8 3431
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/17-Metadata.p  	9 3820
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/18-Metadata.p  	10 4184
/Volumes/Biggy/Discog/artists-allmu

  --> This file is 835.3kB.
Saved data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToVariations.p
  --> This file is 835.3kB.
Current Time is Fri Mar 27, 2020 16:35:24 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 4 seconds.
Current Time is Fri Mar 27, 2020 16:35:24 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 38677 Artists
	Shape --> (38677, 1)
  Finding Real Artist Name
	Shape --> (38677, 3)
  Removing None Artist
	Shape --> (38677, 3)
  Finding Disc Artist Name
	Shape --> (38677, 4)
DataFrame Shape is (38677, 4)
Current Time is Fri Mar 27, 2020 16:35:25 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 1 seconds.
Saving Master Artist DB File
  --> This file is 2.9MB.
Current Tim

,Albums
0000003100,"{'Albums': {'0000476912': 'Ohla Quem Chega', '..."
0000003500,"{'Albums': {}, 'Single/EP': {'0000934205': 'Ge..."
0000006000,{'Albums': {'0000237785': 'Eyes of the Basilis...
0000007300,"{'Albums': {}, 'Album': {'0000089708': 'Motiva..."
0000015100,"{'Albums': {'0000597370': 'Life Rays', '000035..."


#### MusicBrainz

In [4]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discMB)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistMBDB = getSlimArtistDB(discMB, force=True)
discSlimArtistMBDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Mar 15, 2020 12:23:15 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/0-Metadata.p  	0 195
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/1-Metadata.p  	1 410
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/10-Metadata.p  	2 639
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/11-Metadata.p  	3 861
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/12-Metadata.p  	4 1064
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/13-Metadata.p  	5 1273
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/14-Metadata.p  	6 1495
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/15-Metadata.p  	7 1693
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/16-Metadata.p  	8 1913
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/17-Metadata.p  	9 2106
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/18-Metadata.p  	10 2310
/Vol

  --> This file is 920.7kB.
Saved data to /Volumes/Biggy/Discog/db-musicbrainz/ArtistIDToName.p
  --> This file is 920.7kB.
Saving 21038 entries to /Volumes/Biggy/Discog/db-musicbrainz/ArtistIDToVariations.p

Saving data to /Volumes/Biggy/Discog/db-musicbrainz/ArtistIDToVariations.p
  --> This file is 991.6kB.
Saved data to /Volumes/Biggy/Discog/db-musicbrainz/ArtistIDToVariations.p
  --> This file is 991.6kB.
Current Time is Sun Mar 15, 2020 12:23:18 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 3 seconds.
Current Time is Sun Mar 15, 2020 12:23:18 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 21038 Artists
	Shape --> (21038, 1)
  Finding Real Artist Name
	Shape --> (21038, 3)
  Removing None Artist
	Shape --> (21038, 3)
  Finding Disc Artist Name
	Shape --> (21038, 4)
  Cleaning Disc Artist Name (MusicBrainz Only)
	Shape --> (21038, 4)
DataFrame Shape 

,Name,Artist,Num,DiscArtist
148150553282083715092252970758494088400,Kölner Saxophon Mafia,Kölner Saxophon Mafia,None,Kölner Saxophon Mafia
153306575689393285671340715814524277900,Afrodite (Czech group),Afrodite (Czech group),None,Afrodite
164497588684518360574139427720206551000,Pure Prairie League,Pure Prairie League,None,Pure Prairie League
192627206585753480770469159350953417800,The Righteous Brothers,The Righteous Brothers,None,The Righteous Brothers
196485629247468963190624302831920892400,Planeta Fakón,Planeta Fakón,None,Planeta Fakón


#### Discogs

In [24]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discDC)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistDCDB = getSlimArtistDB(discDC, force=True)
discSlimArtistDCDB.head()


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistDCAlbumsDB = getArtistAlbumsDB(discDC, force=True)
discArtistDCAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Mar 20, 2020 16:57:02 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-discogs-db/metadata/0-Metadata.p  	0 7119
/Volumes/Biggy/Discog/artists-discogs-db/metadata/1-Metadata.p  	1 14499
/Volumes/Biggy/Discog/artists-discogs-db/metadata/10-Metadata.p  	2 21665
/Volumes/Biggy/Discog/artists-discogs-db/metadata/11-Metadata.p  	3 28919
/Volumes/Biggy/Discog/artists-discogs-db/metadata/12-Metadata.p  	4 36192
/Volumes/Biggy/Discog/artists-discogs-db/metadata/13-Metadata.p  	5 43522
/Volumes/Biggy/Discog/artists-discogs-db/metadata/14-Metadata.p  	6 50598
/Volumes/Biggy/Discog/artists-discogs-db/metadata/15-Metadata.p  	7 57802
/Volumes/Biggy/Discog/artists-discogs-db/metadata/16-Metadata.p  	8 64930
/Volumes/Biggy/Discog/artists-discogs-db/metadata/17-Metadata.p  	9 72154
/Volumes/Biggy/Discog/artists-discogs-db/metadata/18-Metadata.p  	10 79377
/Volumes/Biggy/Discog/artists-disc

  --> This file is 26.9MB.
Saved data to /Volumes/Biggy/Discog/db-discogs/ArtistIDToVariations.p
  --> This file is 26.9MB.
Current Time is Fri Mar 20, 2020 16:59:05 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 2.0 minutes.
Current Time is Fri Mar 20, 2020 16:59:06 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 718453 Artists
	Shape --> (718453, 1)
  Finding Real Artist Name
	Shape --> (718453, 3)
  Removing None Artist
	Shape --> (718453, 3)
  Finding Disc Artist Name
	Shape --> (718453, 4)
DataFrame Shape is (718453, 4)
Current Time is Fri Mar 20, 2020 16:59:25 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 18 seconds.
Saving Master Artist DB File
  --> This file is 57.5MB.


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


#### AceBootlegs

In [33]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discAB)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistABDB = getSlimArtistDB(discAB, force=True)
discSlimArtistABDB.head()


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistABAlbumsDB = getArtistAlbumsDB(discAB, force=True)
discArtistABAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Mar 27, 2020 16:36:16 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/0-Metadata.p  	0 2
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/10-Metadata.p  	1 3
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/11-Metadata.p  	2 5
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/12-Metadata.p  	3 14
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/13-Metadata.p  	4 16
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/14-Metadata.p  	5 20
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/15-Metadata.p  	6 21
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/16-Metadata.p  	7 24
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/17-Metadata.p  	8 27
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/18-Metadata.p  	9 32
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/19-Metadata.p  	10 33
/Volumes/Biggy/Discog/ar

Creating Pandas DataFrame for 341 Artists
	Shape --> (341, 1)
DataFrame Shape is (341, 1)
Saving Master Artist Albums DB File
  --> This file is 132.2kB.
Current Time is Fri Mar 27, 2020 16:36:17 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
=================================== Creating Artist Albums DB ===================================] took 125.0 millseconds.


,Albums
808801813200,"{'Bootleg': {'10753': 'Warning The Audience', ..."
393858009400,{'Bootleg': {'27966': 'Live Trash'}}
98803225010,{'Bootleg': {'26358': '14 September 1978 El Mo...
363272091111,{'Bootleg': {'19772': 'Live In France'}}
872005022711,{'Bootleg': {'20735': 'Live At The Greek Theat...


#### RateYourMusic

In [31]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discRM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistRMDB = getSlimArtistDB(discRM, force=True)
discSlimArtistRMDB.head()


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistRMAlbumsDB = getArtistAlbumsDB(discRM, force=True)
discArtistRMAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Mar 27, 2020 16:34:02 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/16-Metadata.p  	0 1
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/19-Metadata.p  	1 2
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/20-Metadata.p  	2 3
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/27-Metadata.p  	3 4
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/28-Metadata.p  	4 5
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/45-Metadata.p  	5 6
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/49-Metadata.p  	6 7
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/55-Metadata.p  	7 8
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/57-Metadata.p  	8 9
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/58-Metadata.p  	9 10
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/61-Metadata.p  	10 11
/Volume

,Albums
416,{'Album': {'6364': 'Peter Green's Fleetwood Ma...
146419,{'Single': {'613366': 'Johnny Runs for Paregor...
1090320,"{'Album': {'6238923': 'Great Pine', '9905730':..."
3127,"{'Album': {'16808': 'Kenny G.', '11724': 'G Fo..."
2628,"{'Album': {'9406': 'A Flock of Seagulls', '940..."


#### LastLM

In [35]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discLM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistLMDB = getSlimArtistDB(discLM, force=True)
discSlimArtistLMDB.head()


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistLMAlbumsDB = getArtistAlbumsDB(discLM, force=True)
discArtistLMAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sat Mar 28, 2020 20:19:19 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/0-Metadata.p  	0 453
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/1-Metadata.p  	1 917
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/10-Metadata.p  	2 1409
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/11-Metadata.p  	3 1881
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/12-Metadata.p  	4 2373
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/13-Metadata.p  	5 2891
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/14-Metadata.p  	6 3368
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/15-Metadata.p  	7 3848
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/16-Metadata.p  	8 4339
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/17-Metadata.p  	9 4782
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/18-Metadata.p  	10 5244
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/19-Metad

  --> This file is 1.6MB.
Saved data to /Volumes/Biggy/Discog/db-lastfm/ArtistIDToVariations.p
  --> This file is 1.6MB.
Current Time is Sat Mar 28, 2020 20:19:25 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 6 seconds.
Current Time is Sat Mar 28, 2020 20:19:25 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 48049 Artists
	Shape --> (48049, 1)
  Finding Real Artist Name
	Shape --> (48049, 3)
  Removing None Artist
	Shape --> (48049, 3)
  Finding Disc Artist Name
	Shape --> (48049, 4)
DataFrame Shape is (48049, 4)
Current Time is Sat Mar 28, 2020 20:19:26 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 1 seconds.
Saving Master Artist DB File
  --> This file is 6.2MB.
Current Time is S

,Albums
7371178800,{'Albums': {}}
10197954100,{'Albums': {}}
10998118200,{'Albums': {}}
11060537400,{'Albums': {}}
11350890500,{'Albums': {}}


## Artist ID --> Albums

#### AllMusic

In [6]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Mar 15, 2020 12:25:32 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/0-MediaMetadata.p 	382       382       1778      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/1-MediaMetadata.p 	746       746       3369      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/10-MediaMetadata.p 	1163      1163      5139      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/11-MediaMetadata.p 	1537      1537      6713      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/12-MediaMetadata.p 	1940      1940      8418      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/13-MediaMetadata.p 	2322      2322      10071     
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/14-MediaMetadata.p 	2681      2681      11633     
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/15-MediaMetadata.p 	3039      3039      13278     
/Volumes/Biggy/Discog/artists-al

/Volumes/Biggy/Discog/artists-allmusic-db/metadata/81-MediaMetadata.p 	31344     31344     136855    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/82-MediaMetadata.p 	31711     31711     138450    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/83-MediaMetadata.p 	32131     32131     140258    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/84-MediaMetadata.p 	32508     32508     141938    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/85-MediaMetadata.p 	32910     32910     143887    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/86-MediaMetadata.p 	33291     33291     145488    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/87-MediaMetadata.p 	33715     33715     147220    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/88-MediaMetadata.p 	34113     34113     148866    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/89-MediaMetadata.p 	34511     34511     150573    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/9-MediaMetadata.p 	34903     34

,Albums
0000003100,"{'Albums': {'0000476912': 'Ohla Quem Chega', '..."
0000003500,"{'Albums': {}, 'Single/EP': {'0000934205': 'Ge..."
0000006000,{'Albums': {'0000237785': 'Eyes of the Basilis...
0000007300,"{'Albums': {}, 'Album': {'0000089708': 'Motiva..."
0000015100,"{'Albums': {'0000597370': 'Life Rays', '000035..."


#### MusicBrainz

In [7]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discMB)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Mar 15, 2020 12:26:46 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/0-MediaMetadata.p 	195       0         777       
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/1-MediaMetadata.p 	410       0         1782      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/10-MediaMetadata.p 	639       0         2815      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/11-MediaMetadata.p 	861       0         3684      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/12-MediaMetadata.p 	1064      0         4408      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/13-MediaMetadata.p 	1273      0         5275      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/14-MediaMetadata.p 	1495      0         6041      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/15-MediaMetadata.p 	1693      0         6802      
/Volumes

/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/8-MediaMetadata.p 	16602     0         64917     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/80-MediaMetadata.p 	16836     0         66065     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/81-MediaMetadata.p 	17057     0         66880     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/82-MediaMetadata.p 	17263     0         67665     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/83-MediaMetadata.p 	17474     0         68591     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/84-MediaMetadata.p 	17698     0         69367     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/85-MediaMetadata.p 	17896     0         70226     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/86-MediaMetadata.p 	18093     0         71066     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/87-MediaMetadata.p 	18326     0         72149     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/88

,Albums
148150553282083715092252970758494088400,{'Album': {52614366479835895669985691312690064...
153306575689393285671340715814524277900,{'Album': {21489969636009873249112532561634349...
164497588684518360574139427720206551000,{'Album': {25830796384697956307029700203839095...
192627206585753480770469159350953417800,{'Album': {16465593027634309455140682224063378...
196485629247468963190624302831920892400,{'Album': {18602610412619364467964020782245346...


#### Discogs

In [25]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discDC)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Mar 20, 2020 16:59:40 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-discogs-db/metadata/0-MediaMetadata.p 	7119      3661      19539     
/Volumes/Biggy/Discog/artists-discogs-db/metadata/1-MediaMetadata.p 	14499     7402      40175     
/Volumes/Biggy/Discog/artists-discogs-db/metadata/10-MediaMetadata.p 	21665     11095     60234     
/Volumes/Biggy/Discog/artists-discogs-db/metadata/11-MediaMetadata.p 	28919     14810     80315     
/Volumes/Biggy/Discog/artists-discogs-db/metadata/12-MediaMetadata.p 	36192     18560     100442    
/Volumes/Biggy/Discog/artists-discogs-db/metadata/13-MediaMetadata.p 	43522     22285     120751    
/Volumes/Biggy/Discog/artists-discogs-db/metadata/14-MediaMetadata.p 	50598     25936     140354    
/Volumes/Biggy/Discog/artists-discogs-db/metadata/15-MediaMetadata.p 	57802     29623     160974    
/Volumes/Biggy/Discog/artists-discogs-db

,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


#### AceBootlegs

In [23]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discAB)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Mar 19, 2020 22:10:20 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/0-MediaMetadata.p 	2         0         2         
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/10-MediaMetadata.p 	3         0         3         
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/11-MediaMetadata.p 	5         0         5         
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/12-MediaMetadata.p 	14        0         14        
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/13-MediaMetadata.p 	16        0         16        
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/14-MediaMetadata.p 	20        0         20        
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/15-MediaMetadata.p 	21        0         21        
/Volumes/Biggy/Discog/artists-acebootlegs-db/metadata/16-MediaMetadata.p 	24        0         24        
/Volume

  --> This file is 68.6kB.
Saved data to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToAlbumNames.p
  --> This file is 68.6kB.
Saving 341 entries to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToAlbumRefs.p

Saving data to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToAlbumRefs.p
  --> This file is 78.9kB.
Saved data to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToAlbumRefs.p
  --> This file is 78.9kB.
Saving 341 entries to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToCoreAlbumNames.p

Saving data to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToCoreAlbumNames.p
  --> This file is 4.4kB.
Saved data to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToCoreAlbumNames.p
  --> This file is 4.4kB.
Saving 341 entries to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToCoreAlbumRefs.p

Saving data to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToCoreAlbumRefs.p
  --> This file is 4.4kB.
Saved data to /Volumes/Biggy/Discog/db-acebootlegs/ArtistIDToCoreAlbumRefs.p
  --> This file is 4.4kB.


,Albums
808801813200,"{'Bootleg': {'10753': 'Warning The Audience', ..."
393858009400,{'Bootleg': {'27966': 'Live Trash'}}
98803225010,{'Bootleg': {'26358': '14 September 1978 El Mo...
363272091111,{'Bootleg': {'19772': 'Live In France'}}
872005022711,{'Bootleg': {'20735': 'Live At The Greek Theat...


#### RateYourMusic

In [29]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discRM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Mar 27, 2020 16:33:01 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/16-MediaMetadata.p 	1         0         45        
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/19-MediaMetadata.p 	2         0         48        
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/20-MediaMetadata.p 	3         0         49        
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/27-MediaMetadata.p 	4         0         85        
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/28-MediaMetadata.p 	5         0         113       
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/45-MediaMetadata.p 	6         0         134       
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/49-MediaMetadata.p 	7         0         144       
/Volumes/Biggy/Discog/artists-rateyourmusic-db/metadata/55-MediaMetadata.p 	8         0         1

,Albums
416,{'Album': {'6364': 'Peter Green's Fleetwood Ma...
146419,{'Single': {'613366': 'Johnny Runs for Paregor...
1090320,"{'Album': {'6238923': 'Great Pine', '9905730':..."
3127,"{'Album': {'16808': 'Kenny G.', '11724': 'G Fo..."
2628,"{'Album': {'9406': 'A Flock of Seagulls', '940..."


#### LastLM

In [12]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discLM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Mar 27, 2020 15:29:50 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/0-MediaMetadata.p 	304       304       304       
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/1-MediaMetadata.p 	614       614       614       
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/10-MediaMetadata.p 	959       959       959       
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/11-MediaMetadata.p 	1285      1285      1285      
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/12-MediaMetadata.p 	1632      1632      1632      
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/13-MediaMetadata.p 	1974      1974      1974      
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/14-MediaMetadata.p 	2298      2298      2298      
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/15-MediaMetadata.p 	2634      2634      2634      
/Volumes/Biggy/Discog/artists-lastfm-db/metadata

/Volumes/Biggy/Discog/artists-lastfm-db/metadata/87-MediaMetadata.p 	28933     28933     28933     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/88-MediaMetadata.p 	29302     29302     29302     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/89-MediaMetadata.p 	29656     29656     29656     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/9-MediaMetadata.p 	29968     29968     29968     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/90-MediaMetadata.p 	30293     30293     30293     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/91-MediaMetadata.p 	30576     30576     30576     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/92-MediaMetadata.p 	30887     30887     30887     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/93-MediaMetadata.p 	31214     31214     31214     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/94-MediaMetadata.p 	31538     31538     31538     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/95-MediaMetadata.p 	31857     31857     31857     
/

,Albums
7371178800,{'Albums': {}}
10197954100,{'Albums': {}}
10998118200,{'Albums': {}}
11060537400,{'Albums': {}}
11350890500,{'Albums': {}}


***
***
***

## Artist ID --> Genre, Style, Artists Lookup Table

In [22]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Feb 21, 2020 13:02:48 for Creating Artist DBs



Saving 0 entries to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToGenre.p

Saving data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToGenre.p
  --> This file is 14B.
Saved data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToGenre.p
  --> This file is 14B.
Saving 0 entries to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToStyle.p

Saving data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToStyle.p
  --> This file is 14B.
Saved data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToStyle.p
  --> This file is 14B.
Saving 0 entries to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToCollaborations.p

Saving data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToCollaborations.p
  --> This file is 14B.
Saved data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToCollaborations.p
  --> This file is 14B.
Current Time is Fri Feb 21, 2020 13:02:48 for Done wit

In [23]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discDC)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Feb 21, 2020 13:02:48 for Creating Artist DBs
/Volumes/Biggy/Discog/albums-discogs-db/metadata/0-ArtistMetadata.p 	4771
/Volumes/Biggy/Discog/albums-discogs-db/metadata/1-ArtistMetadata.p 	9481
/Volumes/Biggy/Discog/albums-discogs-db/metadata/10-ArtistMetadata.p 	16165
/Volumes/Biggy/Discog/albums-discogs-db/metadata/11-ArtistMetadata.p 	22936
/Volumes/Biggy/Discog/albums-discogs-db/metadata/12-ArtistMetadata.p 	29738
/Volumes/Biggy/Discog/albums-discogs-db/metadata/13-ArtistMetadata.p 	33792
/Volumes/Biggy/Discog/albums-discogs-db/metadata/14-ArtistMetadata.p 	40028
/Volumes/Biggy/Discog/albums-discogs-db/metadata/15-ArtistMetadata.p 	46750
/Volumes/Biggy/Discog/albums-discogs-db/metadata/16-ArtistMetadata.p 	52022
/Volumes/Biggy/Discog/albums-discogs-db/metadata/17-ArtistMetadata.p 	56703
/Volumes/Biggy/Discog/albums-discogs-db/metadata/18-ArtistMetadata.p 	63415
/Volumes/Biggy

  --> This file is 17.4MB.
Saved data to /Volumes/Biggy/Discog/db-discogs/ArtistIDToStyle.p
  --> This file is 17.4MB.
Saving 572424 entries to /Volumes/Biggy/Discog/db-discogs/ArtistIDToCollaborations.p

Saving data to /Volumes/Biggy/Discog/db-discogs/ArtistIDToCollaborations.p
  --> This file is 37.7MB.
Saved data to /Volumes/Biggy/Discog/db-discogs/ArtistIDToCollaborations.p
  --> This file is 37.7MB.
Current Time is Fri Feb 21, 2020 13:07:38 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 4.8 minutes.


## Album ID --> Name, Ref, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

In [26]:

discArtistLMAlbumsDB = getArtistAlbumsDB(discRM, force=False)

Current Time is Fri Mar 27, 2020 16:13:09 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Current Time is Fri Mar 27, 2020 16:13:09 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
=================================== Creating Artist Albums DB ===================================] took 6.0 millseconds.


In [27]:
from collections import Counter
cntr = Counter()
for artistData in discArtistLMAlbumsDB["Albums"]:
    for key in artistData.keys():
        cntr[key] += 1
cntr

Counter({'Single': 9,
         'V/A Compilation': 9,
         'Album': 10,
         'EP': 7,
         'Appears On': 5,
         'Compilation': 4,
         'Bootleg / Unauthorized': 3,
         'Live Album': 3,
         'Video': 2})

# Pandas DB

## Slim Artist DB

In [ ]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

## Artist DB

In [ ]:
from masterdb import getArtistDB

In [ ]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

## Artist Metadata DB

In [ ]:
from masterdb import getArtistMetadataDB

In [ ]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

## Artist Albums DB

In [ ]:
from masterdb import getArtistAlbumsDB

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=False)
discArtistAlbumsDB.head()

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

## Albums DB

In [ ]:
from masterdb import getAlbumDB

In [ ]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

## Artist Album ID --> Known Albums

In [ ]:
from masterdb import getArtistAlbumKnownDB

In [ ]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

# Joining Artist ID DataFrame

In [ ]:
from masterdb import createMasterDB

In [ ]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

In [ ]:
from random import random